# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import re as re
import ast
import datetime
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('main_task.csv')

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [4]:
# мои данные
# Евгений Бабин
# mail: evgeny.babin.nov@yandex.ru
# Skillfactory: Evgeny_Nov
# slack:  Evgeny_Nov
# kaggle: EvgenyNov, notebook https://www.kaggle.com/evgenynov/notebook7f973bf102
# github: Evgeny-Nov

In [5]:
# проведём небольшое исследование данных

print("******************************************************")
print("количество пустых значений")
print("")
print(df.isna().sum())
print("")
print("******************************************************")
print("типы объектов")
print("")
print(df.dtypes)
print("")
print("******************************************************")
print("уникальные диапазоны цен")
print("")
print(df['Price Range'].unique())
print("")
print("******************************************************")
print("количество ресторанов с данными ценами")
print("")
n1 = len(df[df['Price Range'] == '$$ - $$$'])
n2 = len(df[df['Price Range'] == '$'])
n3 = len(df[df['Price Range'] == '$$$$'])
print(['средние цены', n1], ['низкие цены', n2], ['высокие цены', n3])
print("")
print("******************************************************")
print("количество уникальных городов")
print(len(df['City'].unique()))

******************************************************
количество пустых значений

Restaurant_id            0
City                     0
Cuisine Style         9283
Ranking                  0
Rating                   0
Price Range          13886
Number of Reviews     2543
Reviews                  0
URL_TA                   0
ID_TA                    0
dtype: int64

******************************************************
типы объектов

Restaurant_id         object
City                  object
Cuisine Style         object
Ranking              float64
Rating               float64
Price Range           object
Number of Reviews    float64
Reviews               object
URL_TA                object
ID_TA                 object
dtype: object

******************************************************
уникальные диапазоны цен

['$$ - $$$' nan '$$$$' '$']

******************************************************
количество ресторанов с данными ценами

['средние цены', 18412] ['низкие цены', 6279] ['высок

In [6]:
# ряд функций, на основе которых будем формировать новые числовые признаки

# как видно из исследования существует 3 вида цен;
# данная функция конверирует диапазон цен в число: price range
# $' -> 1, '$$ - $$$' -> 2.5, '$$$$' -> 4, 
# так как '$$ - $$$' соответствует наибольшее число ресторанов, то для nan тоже установим 2.5
def GetOrderedPrice(price_range):
    price_map = {'$': 1.0, '$$ - $$$': 2.5, '$$$$': 4}
    if pd.isna(price_range):
        return 2.5
    else:
        return price_map[price_range]
    
# конверует CuisineStyle (строку) в список кухней
# для пустой кухни возвращается список ['Custom'], кухни которого нет среди существующих
def GetCuisines(CuisineStyle):
    if pd.isna(CuisineStyle):
        return ['Custom']
    splits = re.split(r'\,+', CuisineStyle)
    cuisines = []
    for s in splits:
        cuisines.append(re.split(r"\'+", s)[1])
    return cuisines

# конверует Reviews (строку) в список списков, то есть из строки вида:
# "[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"
# получится список списков: [['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]
def GetReviewsAsList(reviews):
    if pd.isna(reviews):
        return [[],[]]  
    # так как попадаются значения тип nan внутри строки, такие случаи надо специально обрабатывать
    s_nan1 = ", nan"    
    if reviews.find(s_nan1) >= 0:      
        reviews = reviews.replace(s_nan1, ", ''")
    s_nan2 = "nan,"
    if reviews.find(s_nan2) >= 0:      
        reviews = reviews.replace(s_nan2, "'',")         
    ls = ast.literal_eval(reviews)
    return ls

# извлекает из ревью время 1-го (review_index = 0) или 2-го (review_index = 1) ревью:
# возвращается строка с датой, если нет нужной даты - возвращается пустая строка
def GetTimeString(reviews, review_index):
    ls = GetReviewsAsList(reviews)
    if (len(ls) < 2):
        return ""    
    times = ls[1]
    if len(times) < review_index + 1:
        return ""
    else:
        return times[review_index]
    
# на основе существования ревью возвращается число:
# если есть оба ревью возвращается 2, если одно ревью: 1, если ни одного ревью: 0
def GetReviewsNumber(time_review1, time_review2):
    if time_review1 != "" and time_review2 != "":
        return 2
    elif time_review1 != "" or time_review2 != "":        
        return 1
    else:
        return 0 

In [7]:
# проведём исследование времен ревью

# добавим столбцы с временами ревью
df['Review1 Date'] = df['Reviews'].apply(lambda x : GetTimeString(x, 0))
df['Review2 Date'] = df['Reviews'].apply(lambda x : GetTimeString(x, 1))

# конвертируем непустые значения в тип время
review1_dates = df[df['Review1 Date']!='']['Review1 Date'].apply(lambda x : datetime.strptime(x, '%m/%d/%Y'))
review2_dates = df[df['Review2 Date']!='']['Review2 Date'].apply(lambda x : datetime.strptime(x, '%m/%d/%Y'))

# найдем время самого последнего интервью:
if review1_dates.max() > review2_dates.max():
    print(review1_dates.max())
else:
    print(review2_dates.max())

2018-02-26 00:00:00


In [8]:
# дата самого последнего ревью 2018-02-26
# фукция возвращает количество дней до данного ревью от этой даты,
# если ревью нет, функция возвращает -1
def GetDaysFromLast(time_string):
    last_time = datetime(2018, 2, 26, 0, 0)
    if time_string == "":
        return -1
    else:
        time = datetime.strptime(time_string, '%m/%d/%Y') 
        return (last_time-time).days

In [9]:
#  создаём новые числовые признаки:
# 'Ordered Price' - порядковый номер цены, 
# 'Is Price'- есть цена или пустое значение, 
# 'Cuisines Number' - число кухонь, 
# 'Review1 Days', 'Review2 Days' - число дней для 2-х ревью (столбец 'Reviews') от самого последнего ревью (по всем ресторанам) 
# 'Reviews Number' количесвто ревью в столбце 'Reviews'

df['Ordered Price'] = df['Price Range'].apply(GetOrderedPrice)
df['Is Price'] = df['Price Range'].apply(lambda x : 0 if pd.isna(x) else 1)
df['Cuisines Number'] = df['Cuisine Style'].apply(lambda x : len(GetCuisines(x)) )
df['Review1 Days'] = df['Review1 Date'].apply(GetDaysFromLast)
df['Review2 Days'] = df['Review2 Date'].apply(GetDaysFromLast)
df['Reviews Number'] = df[['Review1 Date', 'Review2 Date']].apply(lambda x: GetReviewsNumber(*x), axis=1)


In [10]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Review1 Date,Review2 Date,Ordered Price,Is Price,Cuisines Number,Review1 Days,Review2 Days,Reviews Number
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,12/31/2017,11/20/2017,2.5,1,3,57,98,2
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,07/06/2017,06/19/2016,2.5,0,1,235,617,2
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,01/08/2018,01/06/2018,4.0,1,7,49,51,2
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,,,2.5,0,1,-1,-1,0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,11/18/2017,02/19/2017,2.5,1,3,100,372,2


In [11]:
# заполняем пустые числовые значения 'Number of Reviews' нулями 
df['Number of Reviews'] = df['Number of Reviews'].fillna(0)
# удаляем нечисловые столбцы, кроме Restaurant_id и City
df = df.drop(['Cuisine Style','Price Range','Reviews','URL_TA','ID_TA', 'Review1 Date', 'Review2 Date'], axis = 1)

In [12]:
df.head()

,Restaurant_id,City,Ranking,Rating,Number of Reviews,Ordered Price,Is Price,Cuisines Number,Review1 Days,Review2 Days,Reviews Number
0,id_5569,Paris,5570.0,3.5,194.0,2.5,1,3,57,98,2
1,id_1535,Stockholm,1537.0,4.0,10.0,2.5,0,1,235,617,2
2,id_352,London,353.0,4.5,688.0,4.0,1,7,49,51,2
3,id_3456,Berlin,3458.0,5.0,3.0,2.5,0,1,-1,-1,0
4,id_615,Munich,621.0,4.0,84.0,2.5,1,3,100,372,2


In [13]:
# из городов формируем "dummies" столбцы и добавляем в основную таблицу
cities = pd.get_dummies(df.City)
df = df.join(cities)
# # удаляем нечисловой столбец City
df = df.drop(['City'], axis = 1)
df.head()

,Restaurant_id,Ranking,Rating,Number of Reviews,Ordered Price,Is Price,Cuisines Number,Review1 Days,Review2 Days,Reviews Number,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,id_5569,5570.0,3.5,194.0,2.5,1,3,57,98,2,...,0,0,0,1,0,0,0,0,0,0
1,id_1535,1537.0,4.0,10.0,2.5,0,1,235,617,2,...,0,0,0,0,0,0,1,0,0,0
2,id_352,353.0,4.5,688.0,4.0,1,7,49,51,2,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,3458.0,5.0,3.0,2.5,0,1,-1,-1,0,...,0,0,0,0,0,0,0,0,0,0
4,id_615,621.0,4.0,84.0,2.5,1,3,100,372,2,...,1,0,0,0,0,0,0,0,0,0


In [14]:
# можно проверить тип и пустые значения перед предсказанием
# print(len(df))
# print(df.dtypes)
# df.isna().sum()

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [15]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [16]:
# X.head()
# y.head()

In [17]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [18]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [19]:
# X_train.values, y_train.values

# Создаём, обучаем и тестируем модель

In [20]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [21]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [22]:
y_pred

array([4.005, 3.995, 4.06 , ..., 3.265, 3.515, 4.87 ])

In [23]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21246749999999998
